In [12]:
import json
from haralyzer import HarParser, HarPage, HarEntry

In [24]:
har_file = "sinu.har" # Put the path to your HAR file here
output_file_name = har_file.split('.har')[0] + '.txt'
output_file = open(output_file_name, 'w+') # Create a file to write the results of the analysis

In [25]:
with open(har_file, 'r') as f:
    har_parser = HarParser(json.loads(f.read()))

print(har_parser.pages)

[ID: page_0, URL: https://www.sinu.edu.sb/]


In [26]:
har_page = har_parser.pages[0] # Assuming only one page per HAR File

In [27]:
"""If there are more than one pages in the HAR file, then choose the appropriate page by
   running the below code with the appropriate page id"""
# with open(har_file, 'r') as f:
#     har_page = HarPage('page_0', har_data=json.loads(f.read())) # Put the appropriate page id here

# 1. Report the TTFB and page load times
# 2. Report the total number of HTTP GET requests made to the server

In [28]:
output_file.write(f"Host Name: {har_page.hostname}\n")
output_file.write(f"URL: {har_page.url}\n")
output_file.write(f"Time To First Byte: {har_page.time_to_first_byte} ms\n")
output_file.write(f"Page Load Time: {har_page.page_load_time} ms\n")
output_file.write(f"Total GET Requests made to the Server: {len(har_page.get_requests)}\n")

42

# 3. List the different MIME types observed in the requests

In [29]:
mime_types = []
mime_type_count = {}

for entry in har_page.entries:
    mime_type = entry.response.mimeType # Find the mime type of the reponse
    
    if mime_type not in mime_types:
        # Add to the list of mime types
        mime_types.append(mime_type)
        mime_type_count[mime_type] = 0

    mime_type_count[mime_type] += 1 # Increment count of the mime type

In [30]:
output_file.write(f"\nDifferent MIME Types Observed:\n")
for idx, type in enumerate(mime_types):
    output_file.write(f"\t{idx + 1}. {type}\n")

# 4. What fraction of GET requests correspond to: (a) Images, (b) javascript and (c) HTML and CSS

In [31]:
mime_type_count_ = {"Images" : 0,
                    "Videos" : 0,
                    "JavaScript" : 0,
                    "HTML and CSS" : 0,
                    "Fonts" : 0}

for type_count in mime_type_count:
    if "image" in type_count:
        mime_type_count_["Images"] += mime_type_count[type_count]
    elif "video" in type_count:
        mime_type_count_["Videos"] += mime_type_count[type_count]
    elif "javascript" in type_count:
        mime_type_count_["JavaScript"] += mime_type_count[type_count]
    elif ("html" in type_count) or ("css" in type_count):
        mime_type_count_["HTML and CSS"] += mime_type_count[type_count]
    elif "font" in type_count:
        mime_type_count_["Fonts"] += mime_type_count[type_count]

total_get_requests = len(har_page.get_requests)
mime_type_fraction = {}
for type_count in mime_type_count_:
    mime_type_fraction[type_count] = mime_type_count_[type_count] / (total_get_requests * 0.01)

In [32]:
output_file.write(f"\nFraction of different MIME Types in GET Requests:\n")
for idx, type_count in enumerate(mime_type_fraction.keys()):
    output_file.write(f"\t{idx + 1}. {type_count} : {mime_type_fraction[type_count] : .2f} %\n")

s# 5. What is the total size of the assets downloaded as a part of the responses? What fraction of the size constitute of the images? What is the mean and median size of the images?

In [33]:
total_size = har_page.get_total_size(har_page.entries) / 1e6
total_image_size = har_page.image_size / 1e6
image_fraction = (total_image_size / total_size) * 100

images = har_page.image_files
image_size = []
for image in images:
    image_size.append(har_page.get_total_size([image]))
image_size.sort()

mean_image_size = (total_image_size / mime_type_count_["Images"])

if len(image_size) % 2:
    median_image_size = image_size[int(len(image_size) / 2)]
else:
    median_image_size = (image_size[int(len(image_size) / 2) - 1] + image_size[int(len(image_size) / 2)]) / 2

median_image_size /= 1e6

In [34]:
output_file.write(f"\nTotal Size of Downloaded Assets: {total_size : .4f} MB\n")
output_file.write(f"Total Size of Images Downloaded: {total_image_size : .4f} MB\n")
output_file.write(f"Average Size of Images Downloaded: {mean_image_size : .4f} MB\n")
output_file.write(f"Median Size of Images Downloaded: {median_image_size : .4f} MB\n")

output_file.close()